# Use of FoKL-GPy's symbolic Gaussian process in improved fitting of Langmuir isotherm adsorption model parameters

---

## Background

The Langmuir isotherm is
$\theta_1 = \frac{q}{q_{max}} = \frac{\sigma_1 \mu}{1 + \sigma_1 \mu}$
where
$\mu = \frac{m}{M} = \frac{p}{\sqrt{2 \pi M R T}}$.

Rearranging,
$q = \frac{\sigma_1 q_{max} \mu}{1 + \sigma_1 \mu}$.

For varying pressure $p$ at constant temperature $T$, it is typically assumed that $\sigma_1$ is constant such that
$\sigma_1 \mu = a p$.

This assumption, together with $b \equiv q_{max}$, enable
$\frac{p}{q} = \frac{1}{b}p + \frac{1}{ab}$
which is just $y=mx+b$ such that a linear fit of measurements $q(p)$ provides constant values $a$ and $b$ at constant $T$.

---

## The Case for FoKL-GPy

Examining more closely reveals
$\sigma_1 \mu = \sigma_1 \frac{p}{\sqrt{2 \pi M R T}} \implies a = \frac{\sigma_1}{\sqrt{2 \pi M R T}}$.

Unless reaction rates $\alpha$ and $\nu_1$ are constant with respect to pressure, since $\sigma_1 = \frac{\alpha}{\nu_1}$, the typical assumption that $a$ is constant at $T$ may be improved with the modeling of a Gaussian process (GP). In other words, at constant $T$, assuming $a$ is constant is equivalent to assuming $\sigma_1$ is constant.

Examining the reaction rates as expressed in the relative life,
$\sigma_1 = \frac{\alpha}{\nu_1} = \frac{\exp(\Delta_f G / RT)}{\exp(\Delta_r G / RT)} = \exp(\frac{\Delta_f G - \Delta_r G}{RT})$.

Since Gibbs free energy $G$ is not independent of $p$, and therefore $\sigma_1$ is not constant at constant $T$, $\sigma_1$ may be more accurately modeled with a GP rather than linear fit. Furthermore, since a FoKL-GPy model may be embedded in Pyomo, additional chemistry constraints and an optimization objective may be included, though this is also possible with a linear Langmuir isotherm model.

---

## Outline of Method

Langmuir's mica experiment contains measurements of $q$ for varying $p$ at constant $T$. He conducted this experiment at $T=90\ \text{K}$ and again at $T=155\ \text{K}$. Note the improvement enabled by a GP exists even for data obtained at a single temperature, but variations due to temperature may be captured too since the data is available.

To ensure $\sigma_1$ is positive such that the model has physical meaning, the GP model will be trained on the exponent of $\sigma_1$ since the GP itself may not be constrained to positive values.

That is, the FoKL-GPy model at constant $T$ would be $\Delta \equiv \frac{\Delta_f G - \Delta_r G}{RT} = \ln(\sigma_1) \propto p$. However, since data is available at multiple temperatures, the model will be expanded to $\Delta \propto (T, p)$ for demonstration. With the data limited to only two temperatures, it is not expected for the GP to provide accurate predictions except near $T=90\ \text{K}$ or $T=155\ \text{K}$.

From the measurements, $\sigma_1$ must first be calculated. Recall $q = \frac{\sigma_1 q_{max} \mu}{1 + \sigma_1 \mu}$, which yields $\sigma_1 = \frac{q}{\mu(q_{max}-q)}$. If $q_{max}$ is known, then $\sigma_1$ may be solved for since $q$ is measured and $\mu$ is a known function of $(T, p)$. Therefore, $q_{max}$ must be modeled or measured in some way.

Taking $q_{max}$ to be the $b$ value from the linear fit as found by Langmuir, at least for this benchmark comparison, enables $\sigma_1$ to be calculated. Note it is reasonable to assume $q_{max}$ is independent of $p$ since the driving force of the surface characteristics is $T$. So, despite $b$ being independent of $p$, there is no drawback. The place to improve here would be to measure or model $q_{max}$ in perhaps a more direct or accurate way. Measurements of $q$ after a long time, i.e., $\frac{\partial}{\partial t}(q) \approx 0$, could also provide values of $q_{max}$.

## Implementation of Method

Data from Langmuir's mica experiment, Tables II-VI and X, have been written to *txt* files *N2*, *CH4*, *CO*, *Ar*, and *O2*. For example, the *N2* dataset is as follows.

| $T$     | $p$     | $q_{obs}$ | $b$   | $q_{cal}$ |
|---------|---------|---------|---------|---------|
| 90      | 37.5    | 24.2    | 32.5    | 24.6    |
| "       | 25.7    | 22.7    | "       | 22.4    |
| "       | 4.04    | 7.55    | "       | 8.3     |
| "       | 2.8     | 6.8     | "       | 6.2     |
| 155     | 65.8    | 6.2     | 11.8    | 6.3     |
| "       | 37.5    | 4.8     | "       | 4.7     |
| "       | 3.9     | 0.43    | "       | 0.75    |
| "       | 2.2     | 0.43    | "       | 0.43    |

Note that to fill in values of $q$ for *CO*, *Ar*, and *O2*, where $q_{obs}$ is missing in the Tables, Equation 31 was used with $(p,a,b)$ values as listed in the Tables. Also note $q_{cal}$ is included for benchmark comparison, but is not used for the FoKL-GPy model.

Nomenclature:
| Langmuir | FoKL-GPy |
|---|---|
| $q_{obs}$ | $q \implies$ ```q``` |
| $b$ | $q_{max} \implies$ ```qmax``` |
| $q_{cal}$ | $q_{cal} \implies$ ```qcal``` |

---

Importing modules and defining constants:

In [1]:
import os
import sys
sys.path.append(os.path.join('..', '..', '..'))  # include local path (i.e., this package)

from src.FoKL import FoKLRoutines
from src.FoKL import fokl_to_pyomo
import pyomo.environ as pyo
from pyomo.environ import *
import numpy as np


gases = ['N2', 'CH4', 'CO', 'Ar', 'O2']
M = {'N2': 0.0280134, 'CH4': 0.01604246, 'CO': 0.0280101, 'Ar': 0.039948, 'O2': 0.0319988}  # molecular weight, kg / mol
R = 8.31446261815324  # universal gas constant, J / (K * mol)

Loading the data:

In [2]:
data = {}
for gas in gases:
    data.update({gas: np.loadtxt(os.path.join('data', f"{gas}.txt"), skiprows=1)})

Parsing the data and converting units to SI:

In [3]:
T = {}
p = {}
q = {}
qmax = {}
qcal = {}  # for benchmark comparison
for gas in gases:
    T.update({gas: data[gas][:, 0]})  # K
    p.update({gas: data[gas][:, 1] * 1e5})  # bars to Pa
    q.update({gas: data[gas][:, 2] * 1e-9})  # mm^3 to m^3
    qmax.update({gas: data[gas][:, 3] * 1e-9})  # mm^3 to m^3
    qcal.update({gas: data[gas][:, 4] * 1e-9})  # mm^3 to m^3

Calculating $mu$ and $sigma_1$:

In [4]:
mu = {}
sigma1 = {}
for gas in gases:
    mu.update({gas: p[gas] /np.sqrt(2 * np.pi * M[gas] * R * T[gas])})
    sigma1.update({gas: q[gas] / mu[gas] / (qmax[gas] - q[gas])})

Initializing FoKL-GPy models with continuous basis functions, i.e., ```kernel=1```:

In [5]:
N2 = FoKLRoutines.FoKL(kernel=1)
CH4 = FoKLRoutines.FoKL(kernel=1)
CO = FoKLRoutines.FoKL(kernel=1)
Ar = FoKLRoutines.FoKL(kernel=1)
O2 = FoKLRoutines.FoKL(kernel=1)

models = [N2, CH4, CO, Ar, O2]

Train FoKL-GPy models on $\Delta \propto (T^{-1}, \ln(p))$:

In [6]:
i = 0
Delta = {}
for gas in gases:
    print(f"\nTraining {gas}...")
    Delta.update({gas: np.log(sigma1[gas])})
    models[i].fit([1 / T[gas], np.log(p[gas])], Delta[gas], clean=True)
    i += 1


Training N2...


/home/jacobpatrick/FoKL-GPy/docs/_dev/langmuir_isotherm/../../../src/FoKL/FoKLRoutines.py:299: UserWarning: 'inputs' was transposed. Ignore if more datapoints than input variables, else set 'AutoTranspose=False' to disable.
  warnings.warn("'inputs' was transposed. Ignore if more datapoints than input variables, else set "


[1, -15.130113256172834]
[2, -15.130113256172834]
[2, -15.130113256172834]
[3, -15.130113256172834]

Training CH4...
[1, -8.301225371591523]
[2, -8.301225371591523]
[2, -8.301225371591523]
[3, -8.301225371591523]

Training CO...
[1, -25.984883568453817]
[2, -26.676304920601684]
[2, -227.81855462973454]
[3, -227.81855462973454]
[3, 16.417276234068108]
[4, 10.8721748484174]

Training Ar...
[1, -32.662014404218]
[2, -57.56327732551756]
[2, -239.16038334119014]
[3, -239.16038334119014]
[3, -239.16038334119014]
[4, -239.16038334119014]

Training O2...
[1, -16.12512898630761]
[2, -16.12512898630761]


/home/jacobpatrick/.local/lib/python3.12/site-packages/numpy/core/_methods.py:176: RuntimeWarning: overflow encountered in multiply
  x = um.multiply(x, x, out=x)


[2, -18.33700263911544]
[3, -23.020282072502724]
[3, -33.44299268944647]
[4, -33.44299268944647]
[4, -329.94645215298146]
[4, -340.4700939139391]
[5, -340.4700939139391]
[5, -340.4700939139391]
[5, 22.397270607231572]


Convert to Pyomo:

In [7]:
Deltas = []
for gas in gases:
    Deltas.append(f"{gas}_Delta")  # variable name of Delta per gas in Pyomo

m = fokl_to_pyomo.fokl_to_pyomo(models, [['inv_T', 'ln_p'], ['inv_T', 'ln_p'], ['inv_T', 'ln_p'], ['inv_T', 'ln_p'], ['inv_T', 'ln_p']], Deltas, truescale=True, draws=2)  # Pyomo model

Since a model of $q_{max}$ is being included, training FoKL_GPy models:

In [8]:
N2_qmax = FoKLRoutines.FoKL(kernel=1)
CH4_qmax = FoKLRoutines.FoKL(kernel=1)
CO_qmax = FoKLRoutines.FoKL(kernel=1)
Ar_qmax = FoKLRoutines.FoKL(kernel=1)
O2_qmax = FoKLRoutines.FoKL(kernel=1)

models_qmax = [N2_qmax, CH4_qmax, CO_qmax, Ar_qmax, O2_qmax]

i = 0
for gas in gases:
    print(f"\nTraining {gas} qmax...")
    models_qmax[i].fit(T[gas], qmax[gas], clean=True)  # only T because using b == qmax, where b(T)
    i += 1


Training N2 qmax...
[1, -847.4013565257214]
[2, -847.4013565257214]
[3, -847.4013565257214]
[4, -847.4013565257214]

Training CH4 qmax...
[1, -973.3726442815567]
[2, -973.3726442815567]
[3, -973.3726442815567]
[4, -973.3726442815567]

Training CO qmax...


/home/jacobpatrick/FoKL-GPy/docs/_dev/langmuir_isotherm/../../../src/FoKL/FoKLRoutines.py:1346: RuntimeWarning: divide by zero encountered in log
  lik = -(n / 2) * np.log(siglik) - (n - 1) / 2


[1, -inf]
[2, -inf]
[3, -inf]
[4, -inf]

Training Ar qmax...
[1, -437.8416068361252]
[2, -437.8416068361252]
[3, -437.8416068361252]
[4, -437.8416068361252]

Training O2 qmax...
[1, -632.5911741128716]
[2, -632.5911741128716]
[3, -632.5911741128716]


/home/jacobpatrick/.local/lib/python3.12/site-packages/numpy/core/_methods.py:176: RuntimeWarning: overflow encountered in multiply
  x = um.multiply(x, x, out=x)


[4, -632.5911741128716]


Appending FoKL-GPy models to pre-existing Pyomo model:

In [9]:
# # Convert one-by-one:
# for model_qmax in models_qmax:
#     model_qmax.to_pyomo('T', f"{gas}_qmax", truescale=True, m=m)

# Or, convert all at once with less function calls (preferred):
qmaxs = []
for gas in gases:
    qmaxs.append(f"{gas}_qmax")
fokl_to_pyomo.fokl_to_pyomo(models_qmax, [['T'], ['T'], ['T'], ['T'], ['T']], qmaxs, truescale=True, m=m, draws=2)  # 'T' and '{gas}_qmax' already defined (though wasn't necessary), so 'fokl_to_pyomo' will ignore; but this is needed to embed GP in Pyomo

Expand upon Pyomo model:

1. $\sigma_1 = \exp(\Delta)$
2. $q = \frac{\sigma_1 q_{max} \mu}{\sigma_1 \mu}$
3. $\theta_1 = \frac{q}{q_{max}}$

In [10]:
m.p = pyo.Var(within=pyo.NonNegativeReals, bounds=np.exp(m.ln_p.bounds).tolist(), initialize=np.exp(m.ln_p.bounds[0]))

def eq1(m, i):
    return m.component(f"{gas}_sigma1")[i] == exp(m.component(f"{gas}_Delta")[i])

def eq2(m, i, j):
    return m.component(f"{gas}_q")[i, j] == m.component(f"{gas}_sigma1")[i] * m.component(f"{gas}_qmax")[j] * m.component(f"{gas}_mu") / (1 + m.component(f"{gas}_sigma1")[i] * m.component(f"{gas}_mu"))

def eq3(m, i, j):
    return m.component(f"{gas}_theta1")[i, j] == m.component(f"{gas}_q")[i, j] / m.component(f"{gas}_qmax")[j]

im = 0  # needed to index GP (i.e., model) since multiple draws leads to multiple equations in order to preserve Bayesian statistics; f"GP{im}_scenario" created in 'fokl_to_pyomo'
im2 = 5  # for second set of GP's (qmax)
for gas in gases:

    # Define variables, constant across scenarios:
    for var in ['mu']:  # 'qmax' already defined by additional GP models
        m.add_component(f"{gas}_{var}", pyo.Var(within=pyo.NonNegativeReals, initialize=0.0))
    
    # Define variables per Delta scenarios (i.e., variables that benefit from including Bayesian statistics from multiple draws, but only for Delta):
    for var in ['sigma1']:
        m.add_component(f"{gas}_{var}", pyo.Var(m.component(f"GP{im}_scenarios"), within=pyo.NonNegativeReals, initialize=0.0))  # create var for each scenario
    
    # Define variables per (Delta, qmax) scenarios (i.e., variables that benefit from including Bayesian statistics from multiple draws, but for both Delta and qmax combinations):
    for var in ['q', 'theta1']:
        m.add_component(f"{gas}_{var}", pyo.Var(m.component(f"GP{im}_scenarios"), m.component(f"GP{im2}_scenarios"), within=pyo.NonNegativeReals, initialize=0.0))  # create var for each scenario

    # Define constraints:
    m.add_component(f"{gas}_eq1", pyo.Constraint(m.component(f"GP{im}_scenarios"), rule=eq1))  # FoKL model to relative life, sigma1 = exp(Delta)
    m.add_component(f"{gas}_eq2", pyo.Constraint(m.component(f"GP{im}_scenarios"), m.component(f"GP{im2}_scenarios"), rule=eq2))  # Langmuir isotherm, q = sigma1 * qmax * mu / (1 + sigma1 * mu)
    m.add_component(f"{gas}_eq3", pyo.Constraint(m.component(f"GP{im}_scenarios"), m.component(f"GP{im2}_scenarios"), rule=eq3))  # Langmuir isotherm, theta1 = q / qmax

    im += 1
    im2 += 1

Now, the Pyomo model is ready for any additional chemistry constraints or other additions.

To compare against the benchmark data, $q$ will be evaluated in Pyomo at the $(T, p)$ values.

In [21]:
solver = pyo.SolverFactory('multistart')

results = {}
for gas in gases:
    for i in range(data[gas].shape[0]):
        # m.obj = pyo.Objective(expr=abs(m.T - T[gas][i]) + abs(m.p - p[gas][i]), sense=pyo.minimize)  # set objective for solver to optimize
        m.obj = pyo.Objective(expr=sum(m.component(f"{gas}_q")[0, 0] for gas in gases), sense=pyo.minimize)  # set objective for solver to optimize

        # m.T.fix(T[gas][i])
        # m.p.fix(p[gas][i])

        solver.solve(m, solver='ipopt', suppress_unbounded_warning=True)











        results.update({f"{gas}_q_{i}": m.component(f"{gas}_q")(), f"{gas}_theta1_{i}": m.component(f"{gas}_theta1")()})  # only q is for benchmark comparison, but theta1 may be desired for practical applications

'pyomo.core.base.objective.ScalarObjective'>) on block unknown with a new
Component (type=<class 'pyomo.core.base.objective.ScalarObjective'>). This is
usually indicative of a modelling error. To avoid this warning, use
block.del_component() and block.add_component().


ipopt


ApplicationError: No executable found for solver 'ipopt'

Print results:

In [ ]:
for gas in gases:
    print(f"\nResults for {gas}...")
    for in in len(data[gas].shape[0]):
        print(results[f"{gas}_q_{i}"])

<c id=1> [1] <c/> [THE ADSORPTION OF GASES ON PLANE SURFACES OF GLASS, MICA AND PLATINUM. <br>
&emsp;&nbsp; Irving Langmuir <br>
&emsp;&nbsp; Journal of the American Chemical Society 1918 40 (9), 1361-1403 <br>
&emsp;&nbsp; DOI: 10.1021/ja02242a004](https://pubs.acs.org/doi/epdf/10.1021/ja02242a004#) <br>